# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [6]:
## import libraries

!pip3 install torchtext==0.4.0

# Standard libraries
import math
import string
import pandas as pd
import numpy as np
import json

# Machine learning and deep learning libraries
import tensorflow
import keras
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field, LabelField, Example, Dataset
from torchtext.data.utils import get_tokenizer

# Natural Language Processing (NLP) libraries
import nltk
import spacy
from nltk.corpus import stopwords

# Visualization
import matplotlib.pyplot as plt

In [7]:
## reading in json files

"""
Description of json files
* [train-claims,dev-claims].json: JSON files for the labelled training and development set; 
* evidence.json: JSON file containing a large number of evidence passages (i.e. the “knowledge source”); 
* dev-claims-baseline.json: JSON file containing predictions of a baseline system on the development set;
"""

## relative file paths

## baseline system - will not be used for any training/evaluation
devClaimsBaselineFile='./data/dev-claims-baseline.json'
## use this for model training
trainClaimsFile='./data/train-claims.json'
## use this set for hyperparameter tuning and evaluation metric 
devClaimsFile='./data/dev-claims.json'
## evidence files need to be downloaded through https://drive.google.com/file/d/1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6/view?usp=sharing as it is to big to be uploaded to github
evidenceFile='./data/evidence.json'


tokenizer=get_tokenizer("basic_english")


In [8]:

# Define tokenizer
tokenizer=get_tokenizer("basic_english")
stopWords=set(stopwords.words('english'))
punctuation=string.punctuation

def tokenizeAndStopwordRemoval(text):
    tokens=tokenizer(text)
    filteredTokens=[word for word in tokens if (word not in stopWords) and (word not in punctuation)]
    return filteredTokens


TEXT=torchtext.data.Field(tokenize=tokenizeAndStopwordRemoval,
                          init_token='<sos>',
                          eos_token='<sos>',
                          lower=True)
LABEL = torchtext.data.LabelField(dtype=torch.float)

# Load datasets
def load_json_data(json_file, text_field, label_field):
    with open(json_file, 'r') as file:
        data=json.load(file)
    examples=[]
    for i,claimText in data.items():
        text=claimText['claim_text']
        label=claimText['evidences']
        examples.append(Example.fromlist([text, label], fields=[('text', text_field), ('label', label_field)]))
    return examples

train_examples=load_json_data(trainClaimsFile, TEXT, LABEL)
dev_examples=load_json_data(devClaimsFile, TEXT, LABEL)

train_dataset = Dataset(train_examples, fields=[('text', TEXT), ('label', LABEL)])
dev_dataset = Dataset(dev_examples, fields=[('text', TEXT), ('label', LABEL)])

for i, example in enumerate(dev_dataset.examples):
    if i < 5:
        print(vars(example))

# Build vocabulary
TEXT.build_vocab(train_dataset)
LABEL.build_vocab(train_dataset)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Batchify function
def batchify(dataset, bsz):
    data = TEXT.process([getattr(x, 'text') for x in dataset.examples])
    nbatch = data.size(0) // bsz
    data = data.narrow(0, 0, nbatch * bsz)
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

# Batch sizes
batch_size = 20
eval_batch_size = int(len(dev_dataset)/4)

# Process the datasets

train_data = batchify(train_dataset, batch_size)
dev_data = batchify(dev_dataset, eval_batch_size)

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

{'text': ['[south', 'australia]', 'expensive', 'electricity', 'world'], 'label': ['evidence-67732', 'evidence-572512']}
{'text': ['3', 'per', 'cent', 'total', 'annual', 'global', 'emissions', 'carbon', 'dioxide', 'humans', 'australia', 'prod\xaduces', '1', '3', 'per', 'cent', '3', 'per', 'cent', 'amount', 'emissions', 'reductio\xadn', 'effect', 'global', 'climate'], 'label': ['evidence-996421', 'evidence-1080858', 'evidence-208053', 'evidence-699212', 'evidence-832334']}
{'text': ['means', 'world', '1c', 'warmer', 'pre-industrial', 'times'], 'label': ['evidence-889933', 'evidence-694262']}
{'text': ['“as', 'happens', 'zika', 'may', 'also', 'good', 'model', 'second', 'worrying', 'effect', '—', 'disease', 'mutation'], 'label': ['evidence-422399', 'evidence-702226', 'evidence-286834', 'evidence-472751', 'evidence-641043']}
{'text': ['greenland', 'lost', 'tiny', 'fraction', 'ice', 'mass'], 'label': ['evidence-52981', 'evidence-264761', 'evidence-947243', 'evidence-424102']}


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [9]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        #triu returns the upper triangular part of a matrix (2-D tensor) or batch of matrices (see section below)
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

In [10]:
#triu returns the upper triangular part of a matrix (2-D tensor) or batch of matrices (see section below)
torch.triu(torch.ones(3, 3))

tensor([[1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])

In [11]:
# Masking
def masking():
  sz = 4
  mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
  mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

  return mask

masking()

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [12]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term) #0::2 means starting with index 0, step = 2
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)


c:\Users\julev\anaconda3\envs\Python-3-8\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [13]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

best_val_loss = float("inf")
epochs = 10 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, dev_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()


c:\Users\julev\anaconda3\envs\Python-3-8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\julev\anaconda3\envs\Python-3-8\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  1.63s | valid loss  7.49 | valid ppl  1796.61
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  1.54s | valid loss  4.03 | valid ppl    55.99
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  1.40s | valid loss  4.64 | valid ppl   103.97
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  1.33s | valid loss  6.21 | valid ppl   499.13
--------------------------------------------------------------------------

In [14]:
test_loss = evaluate(best_model, dev_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

model.eval()
output=model(dev_data)
probabilities=torch.nn.functional.softmax(output,dim=-1)
_,predictedIndex=torch.max(probabilities,dim=-1)
# for i in predictedIndex:
#     print(predictedIndex[i])

# print(LABEL.vocab.stoi)
from sklearn.metrics import accuracy_score

vocab = LABEL.vocab.stoi 
reverseVocab = {index: word for word, index in vocab.items()} 
def tensor_to_text(tensor, index_to_word):
    words = [index_to_word[index.item()] for index in tensor if index in index_to_word]
    return ' '.join(words)

# print(reverseVocab)

for i in predictedIndex:
    #print(i)
    words = list(set([reverseVocab[index.item()] for index in i]))
    print(words)


#accuracyScore=accuracy_score()

| End of training | test loss  3.48 | test ppl    32.36
['evidence-1120350', 'evidence-117469', 'evidence-88825', 'evidence-368192', 'evidence-946255']
['evidence-1120350', 'evidence-117469', 'evidence-88825', 'evidence-1017712', 'evidence-921243', 'evidence-368192', 'evidence-28478', 'evidence-1033038']
['evidence-1120350', 'evidence-1017712', 'evidence-88825', 'evidence-368192', 'evidence-28478']
['evidence-1120350', 'evidence-1017712', 'evidence-88825', 'evidence-368192', 'evidence-28478']
['evidence-1120350', 'evidence-117469', 'evidence-1127398', 'evidence-368192', 'evidence-28478']
['evidence-1120350', 'evidence-510852', 'evidence-368192', 'evidence-946255', 'evidence-28478']
['evidence-1120350', 'evidence-117469', 'evidence-88825', 'evidence-510852', 'evidence-1127398', 'evidence-368192', 'evidence-28478']
['evidence-1120350', 'evidence-117469', 'evidence-510852', 'evidence-1127398', 'evidence-368192', 'evidence-94670']
['evidence-1120350', 'evidence-510852', 'evidence-1127398',

In [23]:
pred=model(dev_data)
print(pred)

tensor([[[-1.2988,  5.8004, 11.5261,  ..., -0.6375, -0.4646, -0.9988],
         [-0.5857,  4.8666,  3.1875,  ..., -0.8035, -1.2244, -0.1015],
         [-0.5857,  4.8666,  3.1875,  ..., -0.8035, -1.2244, -0.1015],
         ...,
         [-0.2430,  8.1788,  6.7148,  ..., -0.1870, -0.1522, -0.1997],
         [-0.2430,  8.1788,  6.7148,  ..., -0.1870, -0.1522, -0.1997],
         [-0.2430,  8.1788,  6.7148,  ..., -0.1870, -0.1522, -0.1997]],

        [[-1.2827,  5.8204, 11.6077,  ..., -0.6333, -0.4771, -1.0057],
         [-0.7359,  4.7791,  3.2089,  ..., -0.2573, -0.1604, -0.5213],
         [-0.6207,  4.9285,  2.8170,  ..., -1.0316, -0.8102, -0.2190],
         ...,
         [-0.2424,  8.1814,  6.7173,  ..., -0.1865, -0.1524, -0.1997],
         [-0.2424,  8.1814,  6.7173,  ..., -0.1865, -0.1524, -0.1997],
         [-0.2424,  8.1814,  6.7173,  ..., -0.1865, -0.1524, -0.1997]],

        [[-1.2691,  5.8239, 11.6856,  ..., -0.6301, -0.4852, -1.0046],
         [-1.3420,  5.0198,  2.9108,  ..., -0

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*